Development of a wrapper script for MosaicForecast

In [153]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import mosaicforecast as mf
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Filters

### segmental duplications and clustered sites

Development of `bsm/src/mosaicforecast.py` lead to the following:

* The implementation of `MosaicForecast/MuTect2-PoN_filter.py` is so memory inefficient that a typical MuTect2/TNhaplotyper callset saturates even the swap memory.  Therefore this filter was reimplemented using `bcftools` in the `mosaicforecast.segdup_clustered_filter` function (see `bsm/src/mosaicforecast.py`).
* since `MosaicForecast/MuTect2-PoN_filter.py` also filters for allele frequency, its replacement lead to `mosaicforecast.AF_filter`

### gnomAD filter

The goal is to remove calls from `input.vcf.gz` that are represented in gnomAD with $\mathrm{AF} > 0.001$.  The first implementation based on `bcftools annotate` is very inefficient.  Briefly, the command looks like this:

In [159]:
%%bash
if false; then
    gnomADvcf=/projects/shared/gnomAD/gnomad.genomes.r2.1.1.sites.vcf.bgz
    bcftools annotate --threads 15 -a $gnomADvcf -c 'INFO/AF' -Oz -o output.vcf.gz input.vcf.gz
fi

Even on a tiny `input.vcf.gz` with $<100$ records the process did not finish overnight.  The following tricks will be tested to speed up the calculation:

* remove positions from `$gnomADvcf` that are not contained in `input.vcf.gz`
* the `INFO` field contains many many subfields that it might be the factor that slows down `bcftools annotate`; try removing all subfields other than AF

In [1]:
%connect_info

{
  "shell_port": 52235,
  "iopub_port": 57775,
  "stdin_port": 39603,
  "control_port": 46149,
  "hb_port": 39695,
  "ip": "127.0.0.1",
  "key": "1af31d47-129ae359cb64d92fb2ab62e8",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-b92726bd-bc08-4728-8c1b-e505aeb6f8a5.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
